In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('C:/Users/nerpa\Dropbox (Personal)\Research\discrete_sr\code\python\SPIDER_discrete')

import numpy as np

from commons.weight import *
from utils import save, load
from library import *
from process_library_terms import *

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\nerpa\AppData\Local\Temp\ipykernel_11464\1225006309.py:3: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('C:/Users/nerpa\Dropbox (Personal)\Research\discrete_sr\code\python\SPIDER_discrete')
C:\Users\nerpa\AppData\Local\Temp\ipykernel_11464\1225006309.py:3: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('C:/Users/nerpa\Dropbox (Personal)\Research\discrete_sr\code\python\SPIDER_discrete')


ModuleNotFoundError: No module named 'commons'

In [ ]:
%%prun # profiling

dataset = 'data_bu.npy'
positions, vs, deltat, dims = load(dataset, 4)
deltat = float(deltat)
world_size = np.array([dims[0], dims[0], dims[1]])
print(world_size)
cg_res = 5
data_dict = {}
data_dict['v'] = vs
v_obs = Observable('v', 1)
observables = [v_obs]

# fix random seed
np.random.seed(1)

# initial setup of dataset
kernel_sigma = 0.6
srd = SRDataset(world_size, data_dict, positions, observables, 
                kernel_sigma=kernel_sigma, cg_res=cg_res, deltat=deltat, cutoff=6)

# initialize libraries, domains, and weights
srd.make_libraries(max_complexity=5, max_observables=3, max_rho=2)

dom_width = 10
dom_time = 30
srd.make_domains(ndomains=30, domain_size=[dom_width, dom_width, dom_time], pad=8*kernel_sigma)
srd.make_weights(m=16, qmax=1)
srd.make_library_matrices(debug=False)

save('Q_bu.npy', srd.dxs, srd.libs)

In [ ]:
#_, libs = load('Q_bu.npy', 2)
#_, libs = load('Q_XY_t0.01_eta0.1.npy', 2)
_, libs = load('Q_XY_t0.01_eta0.56.npy', 2)
libs = libs.item()

In [ ]:
from commons.sparse_reg_bf import *
import copy

# for regression we now need to construct a Scaler, Initializer, ModelIterator, and Threshold
scaler0 = Scaler(sub_inds=None, char_sizes=libs[0].col_weights, row_norms=None)
init0 = Initializer(method='combinatorial', start_k=2)
#init0 = Initializer(method='power', start_k=10)
#res0 = Residual(residual_type='fixed_column', anchor_col=0)
res0 = Residual(residual_type='dominant_balance')

iter0 = ModelIterator(max_k=10, backward_forward=True, brute_force=True) # test also boolean toggles
thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=None)
#thres0 = Threshold(threshold_type='information', ic=AIC)
#thres0 = Threshold(threshold_type='jump', gamma=1.5, n_terms=3)

opts = {'scaler': scaler0, 'initializer': init0, 'residual': res0,
        'model_iterator': iter0, 'threshold': thres0}
opts['verbose'] = False
opts1 = copy.deepcopy(opts) # need to be careful to deep copy for the stateful ModelIterator
opts['inhomog'] = False
opts['inhomog_col'] = None
#opts['verbose'] = False
sub_inds1 = list(range(len(libs[1].terms)))
sub_inds1.remove(35) # dt rho[v_i * v_j * v_j]
sub_inds1.remove(21) # rho * dt rho[v_i]
opts1['scaler'] = Scaler(sub_inds=sub_inds1, char_sizes=libs[1].col_weights)
opts1['residual'] = Residual(residual_type='fixed_column', anchor_col=13)
opts1['threshold'] = Threshold(threshold_type='jump', gamma=1.5, n_terms=5)

#opts1['verbose'] = True
opts1['inhomog'] = True
opts1['inhomog_col'] = 13 
    
print("###RANK 0 REGRESSION###")
Xi, lambd, best_term, lambda1 = sparse_reg_bf(libs[0].Q, **opts)
print("###RANK 1 REGRESSION###")
Xi2, lambd2, best_term2, lambda1_2 = sparse_reg_bf(libs[1].Q, **opts1)

for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[0].terms[i]}")
print("Model residual: ", lambd)
print("One-term model residual:", lambda1)
print(f"Best term (#{best_term}):", libs[0].terms[best_term])

for i in range(len(Xi2)):
    if Xi2[i]!=0:
        print(f"[Term {i}] {Xi2[i]} * {libs[1].terms[i]}")
print("Model residual: ", lambd2)
print("One-term model residual:", lambda1_2)
print(f"Best term (#{best_term2}):", libs[1].terms[best_term2])

In [ ]:
from identify_models import *

opts['verbose']=False
opts1['verbose']=False

# note that interleave_identify doesn't work with inhomog or fixed-column residual
opts1['inhomog'] = False
opts1['inhomog_col'] = None
opts1['residual'] = copy.deepcopy(opts['residual'])
opts1['threshold'] = Threshold(threshold_type='jump', gamma=1.5, n_terms=None)

eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[0].Q, libs[1].Q], 
[opts, opts1], [libs[0].terms, libs[1].terms], threshold=1e-1, experimental=True)

In [ ]:
#euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]']
euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'di rho']
#euler_terms = ['dt rho[v_i]', 'dj rho[v_i * v_j]', 'di rho', 'dj^2 rho[v_i]']
#euler_terms = ['dj rho[v_i * v_j]', 'di rho']
#col_numbers = [find_term(libs[1].terms, name) for name in euler_terms]
col_numbers = [13, 36, 38, 32]
Xi, lambd = regress(libs[1].Q, col_numbers, libs[1].col_weights)
for i in range(len(Xi)):
    if Xi[i]!=0:
        print(f"[Term {i}] {Xi[i]} * {libs[1].terms[i]}. (Char. size: {libs[1].col_weights[i]})")
print("Model residual:", lambd)